# CS 412 Machine Learning 2020 

# Assignment 3

100 pts

## Goal 

The goal of this assignment 

*  Introduction to working with text data
*  Gain experience with the Scikit-Learn library
*  Gain experience with Naive Bayes and Logistic Regression

## Dataset

**20 Newsgroup Dataset** is a collection 18846 documents which are about 20 different topics.


## Task
Build naive bayes and logistic regression classifiers with the scikit-learn library function to **classify** the documents about their content topic.

## Submission

Follow the instructions at the end.

# 1) Initialize

First, make a copy of this notebook in your drive

# 2) Load Dataset

The 20 Newsgroup Dataset exist on Scikit-Learn library.

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
train_batch = fetch_20newsgroups(subset='train')
test_batch = fetch_20newsgroups(subset='test')

In [3]:
# target groups you will be dealing with
target_groups = train_batch.target_names
target_groups

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [48]:
# creating training and test sets
train_x =  train_batch["data"]
train_y =  train_batch["target"]
test_x  =  test_batch["data"]
test_y  =  test_batch["target"]

In [49]:
print(train_x[0])

thing subject car organ univ maryland colleg park line wonder anyon could enlighten car saw day sport car look late ear call bricklin door realli small addit front bumper separ rest bodi know anyon tellm model name engin spec year product car made histori whatev info funki look car plea thank il brought neighborhood lerxst


In [6]:
print(target_groups[train_y[0]])

rec.autos


# Preprocess

In [6]:
import re

In [7]:
%%capture
import nltk
nltk.download("stopwords")

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [9]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [10]:
# You will use this function to preprocess your data. If you would like to add another preprocessing step in the function, please add it and mention about it in your report.
def preprocess(text):
  text = re.sub("[\w\d._]+@[^\s]+|[^\s]+\.[^\s]+|[^\s]+-[^\s]+|\d+|[^\w\s]","",text.lower().strip())
  text = ' '.join([stemmer.stem(word) for word in re.findall("\w+",text) if word not in stop_words])
  return text

In [34]:
# Apply <preprocess> function on the training and test set 
for i in range(len(train_x)):
  train_x[i] = preprocess(train_x[i])
  #train_y[i] = preprocess(train_y[i])
for j in range(len(test_x)):
  test_x[j] = preprocess(test_x[j])
  #test_y[j] = preprocess(test_x[j])

# Models

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

## Tune Naive Bayes

In [13]:
# Create a CountVectorizer for NB with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words
vectorizer = CountVectorizer(min_df = 50, max_df = 3000, stop_words = stop_words)

In [25]:
# Vectorize your training and test set
train_x = vectorizer.fit_transform(train_x)
test_x =  vectorizer.transform(test_x)

In [26]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

#Initiate the NB model with required components.
model = MultinomialNB()

#Set the hyperparameter space that will be scanned:
#   alpha = (0.1,0.5,1.0,5.0)
params = {"alpha": [0.1, 0.5, 1.0, 5.0]}

#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
#   cv = 3
#   scoring = "accuracy"
grid = GridSearchCV(model, param_grid = params, scoring = "accuracy", cv = 3)
grid.fit(train_x, train_y)

GridSearchCV(cv=3, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.1, 0.5, 1.0, 5.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [17]:
# show the best score
grid.best_score_

0.8212835269890522

In [18]:
# show the best parameter
grid.best_params_

{'alpha': 0.5}

### Evaluate The Best Model for NB

In [80]:
#Create your NB model with the best parameter set.
mdl_nb = MultinomialNB(alpha = 0.5)
#Fit your model on training set.
mdl_nb.fit(train_x, train_y)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [28]:
# Make predictions on test set
preds = mdl_nb.predict(test_x)

In [30]:
# Show your accuracy on test set
print(accuracy_score(test_y, preds))

0.7538502389803505


## Tune Logistic Regresion

In [50]:
# Create a CountVectorizer for LR with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words
vectorizer = CountVectorizer(min_df = 50, max_df = 3000, stop_words = stop_words)


In [51]:
# Vectorizer your training and test set
train_x = vectorizer.fit_transform(train_x)
test_x =  vectorizer.transform(test_x)

In [39]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#Initiate the LR model:
#   max_iter=2000
model = LogisticRegression(max_iter = 2000)

# Set the hyperparameter space that will be scanned:
#     C = (0.001,0.01,0.1,1)     
params = {"C": [0.001, 0.01, 0.1, 1]}

#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
#   cv = 3
#   scoring = "accuracy"
grid = GridSearchCV(model, param_grid = params, scoring = "accuracy", cv = 3)
grid.fit(train_x, train_y)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=2000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=0)

In [40]:
# show the best score
grid.best_score_

0.8328622117461878

In [41]:
# show the best parameter
grid.best_params_

{'C': 0.1}

### Evaluate The Best Model for Logistic Regression

In [74]:
#Create your LR model with the best parameter set.
mdl_lr = LogisticRegression(C = 0.1)
#Fit your model on training set.
mdl_lr.fit(train_x, train_y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
# Make predictions on test set
preds = mdl_lr.predict(test_x)

In [68]:
# Show your accuracy on test set
print(accuracy_score(test_y, preds))

0.7482740308019118


# Feature Importances

In [84]:
# Find the each category's most important top 3 features (words) for LR model and show with a dataframe
coefs = mdl_lr.coef_.tolist()
words = vectorizer.get_feature_names()
d = {}
for i in coefs:
  coef_word = dict(
      sorted(list(zip(words, i)), reverse = True, key = lambda x: x[1])
  )
print(coef_word)

{'christian': 0.824830265200161, 'order': 0.695967224396894, 'koresh': 0.6182598593459419, 'god': 0.6154718527020452, 'beast': 0.5685385178155848, 'moral': 0.5380017156705214, 'promi': 0.5133259406617453, 'toni': 0.489789422143089, 'kent': 0.48388289407937907, 'biblic': 0.47569401629635055, 'jesus': 0.4369321241250293, 'mr': 0.4297888352441519, 'love': 0.41087746552998156, 'bibl': 0.4039872851123279, 'cult': 0.40068243193148567, 'lord': 0.3940634202619963, 'frank': 0.3935450258903126, 'consid': 0.3850370086155353, 'brian': 0.3799751817168419, 'creation': 0.3481099969824912, 'homosexu': 0.34040699530558693, 'contradict': 0.3397489444105362, 'sandvik': 0.33917175852587317, 'church': 0.3362411235302856, 'ancient': 0.3251442127045402, 'fbi': 0.3234969131529137, 'cathol': 0.32171935343352814, 'religion': 0.3204766756004396, 'beer': 0.31790499352015594, 'dead': 0.3154814744578218, 'object': 0.31382265640478335, 'jim': 0.3091507037652769, 'previous': 0.30575266768788817, 'kill': 0.30429697353

In [ ]:
# It should look like this:
lr_df

alt.atheism comp.graphics  ... talk.politics.misc talk.religion.misc
0     atheist       graphic  ...            theodor          christian
1       keith          imag  ...            clinton              order
2       islam          anim  ...        libertarian             koresh

[3 rows x 20 columns]

In [81]:
# Find the each category's most important top 3 features (words) for NB model and show with a dataframe
coefs = mdl_nb.coef_.flatten().tolist()
words = vectorizer.get_feature_names()
coef_word = dict(
    sorted(list(zip(words, coefs)), reverse = True, key = lambda x: x[1])
)
print(coef_word)

{'god': -4.424456993030999, 'peopl': -4.681536787526679, 'say': -4.814944287442128, 'atheist': -4.9281540501874, 'think': -4.948334107191778, 'moral': -5.016275921461057, 'islam': -5.15937676510173, 'believ': -5.16780378126361, 'exist': -5.253149665587168, 'thing': -5.271754853418203, 'make': -5.293907658059336, 'christian': -5.380773591080998, 'mani': -5.468261045204114, 'mean': -5.468261045204114, 'religion': -5.468261045204114, 'time': -5.495289717592033, 'see': -5.523069281699109, 'argument': -5.5352151800012175, 'post': -5.5352151800012175, 'system': -5.555792037689977, 'even': -5.568344503761097, 'keith': -5.598261264799089, 'well': -5.598261264799089, 'seem': -5.606975916901533, 'could': -5.624636420053483, 'said': -5.651726112700148, 'atheism': -5.656313276707054, 'must': -5.660921579793248, 'reason': -5.660921579793248, 'claim': -5.6655512176919895, 'way': -5.684287128749459, 'evid': -5.708211714834705, 'someth': -5.742697890905874, 'object': -5.752773542894616, 'us': -5.77841

In [ ]:
# It should look like this:
nb_df

alt.atheism comp.graphics  ... talk.politics.misc talk.religion.misc
0         god          imag  ...              peopl                god
1       peopl          file  ...                 go          christian
2         say       graphic  ...              think              peopl

[3 rows x 20 columns]

# **Notebook & Report**

Notebook: We may just look at your notebook results; so make sure each cell is run and outputs are there.

Report: Write an at most 1/2 page summary of your approach to this problem at the end of your notebook; this should be like an abstract of a paper or the executive summary.

Must include statements such as:

( Include the problem definition: 1-2 lines )

(Talk about any preprocessing you did, explain your reasoning)

(Talk about train/test sets, size and how split)

(State what your test results are with the chosen method, parameters: e.g. "We have obtained the best results with the ….. classifier (parameters=....) , giving classification accuracy of …% on test data….")

(Comment on feature importances of models)

(Comment on anything that you deem important/interesting)


You will get full points from here as long as you have a good (enough) summary of your work, regardless of your best performance or what you have decided to talk about in the last few lines.



# **Write your report in this cell**

The problem definition is to identify the topic of given documents. For this purpose, two different approaches as naive bayes and logistic regression classifier have been used.

 As preprocessing, the stop words, meaningless words and punctuation marks have been removed using some pre-formed data about stop words. Because they do not give any clue about the topic of the document, they are like noisy data points.

 In total, there are 18846 documents. 11314 of them are used for training and 7532 of them are used for testing as pre-splitted.

 In logistic regression I got 0.7482740308019118 as accuracy using the best hyperparameter (C = 0.1). On the other hand, in naive bayes I got 0.7538502389803505 as the accuracy using the best hyperparameter (alpha = 0.5). Naive bayes is better than the logistic regression in this case with almost 1% difference.

 Finding the best parameters using GridSearchCV took more time in logistic regression compared to the naive bayes.


# **Submission**
You will submit this homework via SUCourse.


Please read this document again before submitting it.

Please submit your **"share link" INLINE in Sucourse submissions.** That is we should be able to click on the link and go there and run (and possibly also modify) your code.

For us to be able to modify, in case of errors etc, you should get your "share link" as **share with anyone in edit mode** 

Download the **.ipynb and the .html** file and upload both of them to Sucourse.
 
Please do your assignment individually, do not copy from a friend or the Internet. Plagiarized assignments will receive -100.
